In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
import numpy as np
from tqdm import tqdm
from sonar.inference_pipelines.text import TextToEmbeddingModelPipeline

t2vec_model = TextToEmbeddingModelPipeline(encoder="text_sonar_basic_encoder", tokenizer="text_sonar_basic_encoder", device=device)
pad = t2vec_model.predict(['End of text.'], source_lang="eng_Latn")[0]

In [ ]:
def pad_tensor(tensor, max_len=20):
    seq = tensor.size(0)
    if seq < max_len:
        padding_needed = max_len - seq
        padding = pad.expand(padding_needed, -1).to(device)
        tensor = torch.cat([tensor, padding], dim=0)
    return tensor[:max_len]


#fineweb[xxx] are my text data files, you should use your own text dataset
for i in tqdm([':100', '100:200', '200:300', '300:400', '400:500', '500:600', '600:700', '700:800', '800:900', '900:']):
    data = torch.load('fineweb['+str(i)+'].pth')
    
    if i==':100':
        n='100'
    elif i=='900:':
        n='900'
    else:
        n=i[-3:]
        data=data[:-2000]
        
    tensor_list_padded = torch.stack([pad_tensor(tensor.to(device)) for tensor in data])
    
    np.save('/workspace/eloise/sentemb/data/'+n+'k.npy', tensor_list_padded.numpy())

Test file creation

In [ ]:
import torch
dataset = torch.load('fineweb[900:].pth')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
from tqdm import tqdm
from sonar.inference_pipelines.text import TextToEmbeddingModelPipeline
t2vec_model = TextToEmbeddingModelPipeline(encoder="text_sonar_basic_encoder", tokenizer="text_sonar_basic_encoder", device=device)

test_sonar = dataset[-1000:]

test_sonar = [sous_liste[:20] for sous_liste in test_sonar]

prompt = []
output = []

for sample in test_sonar:
    prompt.append(sample[0])
    output.append(sample[1:])
    
prompt = torch.stack(prompt).unsqueeze(1)

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
dataset = torch.load('fineweb_random.pth')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

jasper = SentenceTransformer("infgrad/jasper_en_vision_language_v1",
    trust_remote_code=True,
    device=device,
    model_kwargs={"torch_dtype":  torch.bfloat16 if device == torch.device('cuda') else torch.float32},
)
jasper.max_seq_length = 1024

data_jasper = dataset[-1000:][:10]

data_jasper = [sous_liste[1:20] for sous_liste in data_jasper]

test_jasper = []
for sample in data_jasper:
    enc = jasper.encode(sample)
    test_jasper.append(torch.from_numpy(enc).to(device))

In [ ]:
torch.save([prompt, output, test_jasper], '/workspace/eloise/sentemb/data/test_sonarprompt_sonaroutput_jasperoutput.pth')